## Census Tract-level data

In [1]:
# setup
from census import Census
from us import states
import pandas as pd 
import numpy as np
import os


In [ ]:

# Tract-level data
#   can also serve as control totals 

# get HH Size by tract
your_key = "CENSUS_API_KEY" # use environmnet variable or your key here
api_key = os.environ.get('CENSUS_API_KEY', your_key) # fallback to your_key
c = Census(api_key,year=2022) # my API key


In [3]:
tracts = c.acs5.get(('NAME','GEO_ID','B11016_001E','B11016_003E','B11016_004E','B11016_005E','B11016_006E','B11016_007E','B11016_008E','B11016_010E','B11016_011E','B11016_012E','B11016_013E','B11016_014E','B11016_015E','B11016_016E','B25010_001E','B01001_001E',
                    'B08202_001E','B08202_002E','B08202_003E','B08202_004E','B08202_005E','B19326_001E',
                    'B19001_002E','B19001_003E','B19001_004E','B19001_005E','B19001_006E','B19001_007E','B19001_008E','B19001_009E','B19001_010E','B19001_011E','B19001_012E','B19001_013E','B19001_014E','B19001_015E','B19001_016E','B19001_017E',
                    'B26001_001E','B26101_067E','B26101_157E',
                    'B08201_002E','B08201_003E','B08201_004E','B08201_005E','B08201_006E',
                    'B08201_008E','B08201_009E','B08201_010E','B08201_011E','B08201_012E','B08201_014E','B08201_015E','B08201_016E','B08201_017E','B08201_018E','B08201_020E','B08201_021E','B08201_022E','B08201_023E','B08201_024E','B08201_026E','B08201_027E','B08201_028E','B08201_029E','B08201_030E'), 
                     geo={'for': 'tract:*',
                       'in': 'state:{} county:057'.format(states.WA.fips)})

tracts2 = c.acs5st.get(('GEO_ID','S1901_C01_013E'), 
                     geo={'for': 'tract:*',
                       'in': 'state:{} county:057'.format(states.WA.fips)})

tracts = pd.DataFrame(tracts)
tracts2 = pd.DataFrame(tracts2)

# fix geoid
tracts['GEOID2'] = tracts['GEO_ID'].str.slice(start=9)
tracts2['GEOID2'] = tracts2['GEO_ID'].str.slice(start=9)

tracts = tracts.merge(tracts2, on='GEOID2')

tracts = tracts.rename(columns={
    'B11016_001E': 'HH_Total',
    'B11016_010E': 'HH1',
    'B25010_001E': 'HH_AvgSize',
    'B19326_001E': 'Inc_Median',
    'S1901_C01_013E': 'Inc_Mean',
    'B08202_001E': 'Wrk_Total',
    'B08202_002E': 'Wrk0',
    'B08202_003E': 'Wrk1',
    'B08202_004E': 'Wrk2',
    'B08202_005E': 'Wrk3plus',
    'B26001_001E': 'GQ',
    'B26101_067E': 'GQ_Institutionalized',
    'B26101_157E': 'GQ_Noninstitutionalized',
    'B08201_002E': 'Veh0',
    'B08201_003E': 'Veh1',
    'B08201_004E': 'Veh2',
    'B08201_005E': 'Veh3',
    'B08201_006E': 'Veh4plus',
    'B08201_008E': 'HH1VEH0',
    'B08201_009E': 'HH1VEH1',
    'B08201_010E': 'HH1VEH2',
    'B08201_011E': 'HH1VEH3',
    'B08201_012E': 'HH1VEH4',
    'B08201_014E': 'HH2VEH0',
    'B08201_015E': 'HH2VEH1',
    'B08201_016E': 'HH2VEH2',
    'B08201_017E': 'HH2VEH3',
    'B08201_018E': 'HH2VEH4',
    'B08201_020E': 'HH3VEH0',
    'B08201_021E': 'HH3VEH1',
    'B08201_022E': 'HH3VEH2',
    'B08201_023E': 'HH3VEH3',
    'B08201_024E': 'HH3VEH4',
    'B08201_026E': 'HH4VEH0',
    'B08201_027E': 'HH4VEH1',
    'B08201_028E': 'HH4VEH2',
    'B08201_029E': 'HH4VEH3',
    'B08201_030E': 'HH4VEH4',
})


In [4]:
HH4pHH = tracts.apply(lambda row: row['B11016_005E'] + row['B11016_013E'] + row['B11016_006E'] + row['B11016_014E'] + row['B11016_007E'] + row['B11016_015E'] + row['B11016_008E'] + row['B11016_016E'], axis=1)
HH4pPop = tracts.apply(lambda row: ((row['B11016_005E'] + row['B11016_013E']) * 4) + ((row['B11016_006E'] + row['B11016_014E']) * 5) + ((row['B11016_007E'] + row['B11016_015E']) * 6) + ((row['B11016_008E'] + row['B11016_016E']) * 7.5), axis=1)
HH4pAvg = HH4pPop.sum() / HH4pHH.sum()
print("Avg size of 4+ person HHs: ",HH4pAvg)


Avg size of 4+ person HHs:  4.736502989977259


In [5]:

# combine family and non-family households by size
tracts['HH2'] = tracts.apply(lambda row: row['B11016_003E'] + row['B11016_011E'], axis=1)
tracts['HH3'] = tracts.apply(lambda row: row['B11016_004E'] + row['B11016_012E'], axis=1)
tracts['HH4plus'] = tracts.apply(lambda row: row['B11016_005E'] + row['B11016_006E'] + row['B11016_007E'] + row['B11016_008E'] + row['B11016_013E'] + row['B11016_014E'] + row['B11016_015E'] + row['B11016_016E'], axis=1)

# For HH submodel size and income joint distribution
# A two-dimensional distribution table for household size and income was generated using 2017-2021 5-Year ACS data (see Table 19).
# TABLE 19: TWO DIMENSIONAL 2022 HOUSEHOLD SIZE & HOUSEHOLD INCOME DISTRIBUTION
# HOUSEHOLD SIZE x Income level: <25K, 25K≤INCOME<50K, 50K≤INCOME<75K, 75K≤INCOME
# B19001_002E	Less than $10,000  cat 1
# B19001_003E	$10,000 to $14,999 cat 1
# B19001_004E	$15,000 to $19,999 cat 1
# B19001_005E	$20,000 to $24,999 cat 1
# B19001_006E	$25,000 to $29,999 cat 2
# B19001_007E	$30,000 to $34,999 cat 2
# B19001_008E	$35,000 to $39,999 cat 2
# B19001_009E	$40,000 to $44,999 cat 2
# B19001_010E	$45,000 to $49,999 cat 2
# B19001_011E	$50,000 to $59,999 cat 3
# B19001_012E	$60,000 to $74,999 cat 3
# B19001_013E	$75,000 to $99,999 cat 4
# B19001_014E	$100,000 to $124,999 cat 4
# B19001_015E	$125,000 to $149,999 cat 4
# B19001_016E	$150,000 to $199,999 cat 4
# B19001_017E	$200,000 or more cat 4

tracts['INC1'] = tracts.apply(lambda row: row['B19001_002E'] + row['B19001_003E'] + row['B19001_004E'] + row['B19001_005E'], axis=1)                      # < 25k
tracts['INC2'] = tracts.apply(lambda row: row['B19001_006E'] + row['B19001_007E'] + row['B19001_008E'] + row['B19001_009E'] + row['B19001_010E'], axis=1) # 25k-50K
tracts['INC3'] = tracts.apply(lambda row: row['B19001_011E'] + row['B19001_012E'], axis=1)                                                                # 50k-75K
tracts['INC4'] = tracts.apply(lambda row: row['B19001_013E'] + row['B19001_014E'] + row['B19001_015E'] + row['B19001_016E'] + row['B19001_017E'], axis=1) # >75K

# Limit vehicles to number of persons
tracts['VehChoice0'] = tracts['HH1VEH0'] + tracts['HH2VEH0'] + tracts['HH3VEH0'] + tracts['HH4VEH0']
tracts['VehChoice1'] = tracts['HH1VEH1'] + tracts['HH1VEH2'] + tracts['HH1VEH3'] + tracts['HH1VEH4'] + tracts['HH2VEH1'] + tracts['HH3VEH1'] + tracts['HH4VEH1']
tracts['VehChoice2'] = tracts['HH2VEH2'] + tracts['HH2VEH3'] + tracts['HH2VEH4'] + tracts['HH3VEH2'] + tracts['HH4VEH2']
tracts['VehChoice3'] = tracts['HH3VEH3'] + tracts['HH3VEH4'] + tracts['HH4VEH3'] + tracts['HH4VEH4']

# proportions
tracts['HH_Size1'] = tracts['HH1'] / tracts['HH_Total']
tracts['HH_Size2'] = tracts['HH2'] / tracts['HH_Total']
tracts['HH_Size3'] = tracts['HH3'] / tracts['HH_Total']
tracts['HH_Size4plus'] = tracts['HH4plus'] / tracts['HH_Total']

tracts['Worker0_prop'] = tracts['Wrk0'] / tracts['Wrk_Total']
tracts['Worker1_prop'] = tracts['Wrk1'] / tracts['Wrk_Total']
tracts['Worker2_prop'] = tracts['Wrk2'] / tracts['Wrk_Total']
tracts['Worker3_prop'] = tracts['Wrk3plus'] / tracts['Wrk_Total']

tracts['Vehicle0_prop'] = tracts['Veh0'] / tracts['HH_Total']
tracts['Vehicle1_prop'] = tracts['Veh1'] / tracts['HH_Total']
tracts['Vehicle2_prop'] = tracts['Veh2'] / tracts['HH_Total']
tracts['Vehicle3_prop'] = (tracts['Veh3'] + tracts['Veh4plus']) / tracts['HH_Total']

tracts = tracts.drop(columns=['B11016_003E','B11016_004E','B11016_005E','B11016_006E','B11016_007E','B11016_008E','B11016_011E','B11016_012E','B11016_013E','B11016_014E','B11016_015E','B11016_016E','B01001_001E',
                              'B19001_002E','B19001_003E','B19001_004E','B19001_005E','B19001_006E','B19001_007E','B19001_008E','B19001_009E','B19001_010E','B19001_011E','B19001_012E','B19001_013E','B19001_014E','B19001_015E','B19001_016E','B19001_017E',])

# 'B11016_001E', 'B11016_010E', 'B25010_001E', 'B19326_001E',

tracts.to_csv('tracts.csv', index=False)



In [6]:
tracts['HH_AvgSize2'] = round(tracts['HH_AvgSize'],1)
# size_pivot = tracts.pivot_table(index='HH_AvgSize2',columns=['HH1','HH2','HH3','HH4plus'],aggfunc='sum')

size_pivot = tracts.groupby('HH_AvgSize2')[['HH1','HH2','HH3','HH4plus']].agg('sum')
size_pivot.to_csv('HHSize_pivot.csv')


In [7]:

# blockgroup-level data

# get HH Size by BG
bgs = c.acs5.get(('NAME','GEO_ID','B11016_001E','B11016_003E','B11016_004E','B11016_005E','B11016_006E','B11016_007E','B11016_008E','B11016_010E','B11016_011E','B11016_012E','B11016_013E','B11016_014E','B11016_015E','B11016_016E','B25010_001E','B01001_001E','B08006_001E','B19326_001E',
                    'B19001_002E','B19001_003E','B19001_004E','B19001_005E','B19001_006E','B19001_007E','B19001_008E','B19001_009E','B19001_010E','B19001_011E','B19001_012E','B19001_013E','B19001_014E','B19001_015E','B19001_016E','B19001_017E'), 
                     geo={'for': 'block group:*',
                       'in': 'state:{} county:057'.format(states.WA.fips)})

bgs = pd.DataFrame(bgs)

# fix geoid
bgs['GEOID2'] = bgs['GEO_ID'].str.slice(start=9)

# combine family and non-family households by size
bgs['HH_Total'] = bgs['B11016_001E']
bgs['HH1'] = bgs['B11016_010E']
bgs['HH2'] = bgs.apply(lambda row: row['B11016_003E'] + row['B11016_011E'], axis=1)
bgs['HH3'] = bgs.apply(lambda row: row['B11016_004E'] + row['B11016_012E'], axis=1)
bgs['HH4plus'] = bgs.apply(lambda row: row['B11016_005E'] + row['B11016_006E'] + row['B11016_007E'] + row['B11016_008E'] + row['B11016_013E'] + row['B11016_014E'] + row['B11016_015E'] + row['B11016_016E'], axis=1)
bgs['HH_AvgSize'] = bgs['B25010_001E']
# bgs['Wrk_Total'] = bgs['B08006_001E']
# bgs['Wrk_Average'] = bgs.apply(lambda row: 0 if row['HH_Total'] == 0 else row['B08006_001E'] / row['HH_Total'], axis=1) # total workers / total households 
bgs['Inc_Median'] = bgs['B19326_001E']



# For HH submodel size and income joint distribution
# A two-dimensional distribution table for household size and income was generated using 2017-2021 5-Year ACS data (see Table 19).
# TABLE 19: TWO DIMENSIONAL 2022 HOUSEHOLD SIZE & HOUSEHOLD INCOME DISTRIBUTION
# HOUSEHOLD SIZE x Income level: <25K, 25K≤INCOME<50K, 50K≤INCOME<75K, 75K≤INCOME
# B19001_002E	Less than $10,000  cat 1
# B19001_003E	$10,000 to $14,999 cat 1
# B19001_004E	$15,000 to $19,999 cat 1
# B19001_005E	$20,000 to $24,999 cat 1
# B19001_006E	$25,000 to $29,999 cat 2
# B19001_007E	$30,000 to $34,999 cat 2
# B19001_008E	$35,000 to $39,999 cat 2
# B19001_009E	$40,000 to $44,999 cat 2
# B19001_010E	$45,000 to $49,999 cat 2
# B19001_011E	$50,000 to $59,999 cat 3
# B19001_012E	$60,000 to $74,999 cat 3
# B19001_013E	$75,000 to $99,999 cat 4
# B19001_014E	$100,000 to $124,999 cat 4
# B19001_015E	$125,000 to $149,999 cat 4
# B19001_016E	$150,000 to $199,999 cat 4
# B19001_017E	$200,000 or more cat 4

bgs['INC1'] = bgs.apply(lambda row: row['B19001_002E'] + row['B19001_003E'] + row['B19001_004E'] + row['B19001_005E'], axis=1)                      # < 25k
bgs['INC2'] = bgs.apply(lambda row: row['B19001_006E'] + row['B19001_007E'] + row['B19001_008E'] + row['B19001_009E'] + row['B19001_010E'], axis=1) # 25k-50K
bgs['INC3'] = bgs.apply(lambda row: row['B19001_011E'] + row['B19001_012E'], axis=1)                                                                # 50k-75K
bgs['INC4'] = bgs.apply(lambda row: row['B19001_013E'] + row['B19001_014E'] + row['B19001_015E'] + row['B19001_016E'] + row['B19001_017E'], axis=1) # >75K

# proportions
bgs['HH_Size1'] = bgs['HH1'] / bgs['HH_Total']
bgs['HH_Size2'] = bgs['HH2'] / bgs['HH_Total']
bgs['HH_Size3'] = bgs['HH3'] / bgs['HH_Total']
bgs['HH_Size4plus'] = bgs['HH4plus'] / bgs['HH_Total']

bgs = bgs.drop(columns=['B11016_001E','B11016_003E','B11016_004E','B11016_005E','B11016_006E','B11016_007E','B11016_008E','B11016_010E','B11016_011E','B11016_012E','B11016_013E','B11016_014E','B11016_015E','B11016_016E','B25010_001E','B01001_001E','B08006_001E','B19326_001E',
                              'B19001_002E','B19001_003E','B19001_004E','B19001_005E','B19001_006E','B19001_007E','B19001_008E','B19001_009E','B19001_010E','B19001_011E','B19001_012E','B19001_013E','B19001_014E','B19001_015E','B19001_016E','B19001_017E'])

bgs.to_csv('blockgroups.csv', index=False)

In [8]:
bgs['HH_AvgSize2'] = round(bgs['HH_AvgSize'],1)
# size_pivot = tracts.pivot_table(index='HH_AvgSize2',columns=['HH1','HH2','HH3','HH4plus'],aggfunc='sum')

size_pivot = bgs.groupby('HH_AvgSize2')[['HH1','HH2','HH3','HH4plus']].agg('sum')
size_pivot.to_csv('HHSize_pivot_bg.csv')


In [9]:
# HH Size in adjacent counties

# get HH Size by BG
bgs_island = c.acs5.get(('NAME','GEO_ID','B11016_001E','B11016_003E','B11016_004E','B11016_005E','B11016_006E','B11016_007E','B11016_008E','B11016_010E','B11016_011E','B11016_012E','B11016_013E','B11016_014E','B11016_015E','B11016_016E','B25010_001E','B01001_001E','B08006_001E','B19326_001E',
                    'B19001_002E','B19001_003E','B19001_004E','B19001_005E','B19001_006E','B19001_007E','B19001_008E','B19001_009E','B19001_010E','B19001_011E','B19001_012E','B19001_013E','B19001_014E','B19001_015E','B19001_016E','B19001_017E'), 
                     geo={'for': 'block group:*',
                       'in': 'state:{} county:029'.format(states.WA.fips)})

bgs_whatcom = c.acs5.get(('NAME','GEO_ID','B11016_001E','B11016_003E','B11016_004E','B11016_005E','B11016_006E','B11016_007E','B11016_008E','B11016_010E','B11016_011E','B11016_012E','B11016_013E','B11016_014E','B11016_015E','B11016_016E','B25010_001E','B01001_001E','B08006_001E','B19326_001E',
                    'B19001_002E','B19001_003E','B19001_004E','B19001_005E','B19001_006E','B19001_007E','B19001_008E','B19001_009E','B19001_010E','B19001_011E','B19001_012E','B19001_013E','B19001_014E','B19001_015E','B19001_016E','B19001_017E'), 
                     geo={'for': 'block group:*',
                       'in': 'state:{} county:073'.format(states.WA.fips)})

bgs1 = pd.DataFrame(bgs_island)
bgs2 = pd.DataFrame(bgs_whatcom)
bgs = pd.concat([bgs1,bgs2])

# fix geoid
bgs['GEOID2'] = bgs['GEO_ID'].str.slice(start=9)

# combine family and non-family households by size
bgs['HH_Total'] = bgs['B11016_001E']
bgs['HH1'] = bgs['B11016_010E']
bgs['HH2'] = bgs.apply(lambda row: row['B11016_003E'] + row['B11016_011E'], axis=1)
bgs['HH3'] = bgs.apply(lambda row: row['B11016_004E'] + row['B11016_012E'], axis=1)
bgs['HH4plus'] = bgs.apply(lambda row: row['B11016_005E'] + row['B11016_006E'] + row['B11016_007E'] + row['B11016_008E'] + row['B11016_013E'] + row['B11016_014E'] + row['B11016_015E'] + row['B11016_016E'], axis=1)
bgs['HH_AvgSize'] = bgs['B25010_001E']

bgs['HH_AvgSize2'] = round(bgs['HH_AvgSize'],1)
# size_pivot = tracts.pivot_table(index='HH_AvgSize2',columns=['HH1','HH2','HH3','HH4plus'],aggfunc='sum')

size_pivot = bgs.groupby('HH_AvgSize2')[['HH1','HH2','HH3','HH4plus']].agg('sum')
size_pivot.to_csv('HHSize_added_pivot_bg.csv')


## HH Income Index (by tract)

In [10]:
area_income = (tracts['Inc_Mean'] * tracts['HH_Total']).sum() # 5,262,863,780.0
area_avg_income = area_income / (tracts['HH_Total'].sum()) # dived by 50,824 HH = 103,550
tracts['INC_INDEX'] = tracts['Inc_Mean'] / area_avg_income
tracts['INC_INDEX2'] = round(tracts['INC_INDEX'],1)

income_pivot = tracts.groupby('INC_INDEX2')[['INC1','INC2','INC3','INC4']].agg('sum')
income_pivot.to_csv('HHIncome_pivot.csv')

## PUMS HH Size x Income crosstab

In [11]:
# PUMS 2018-22 5 year household data
pums = pd.read_csv('data/PUMS/psam_h53.csv')
len(pums)

173246

In [12]:
skagit_pums = pums[pums['PUMA10'] == 10200]
len(skagit_pums)

8731

In [13]:
skagit_pums['WGTP'].sum()

np.int64(89501)

In [14]:
86510 + 18001 + 129480

233991

In [15]:
35498 + 8654 + 50824

94976

**PUMS 2018-2022**
8,731 HH records in Skagit/Island/San Juan PUMA
89,501 weighted HH

**ACS 5 year 2018-2022**
Island 35,498 households
San Juan 8,654 households
Skagit 50,824 households
Total: 94,976 households

Island 86,510 pop
San Juan 18,001 pop
Skagit 129,480 pop
Total: 233,991 population



In [16]:
pums['NP'].value_counts()

NP
2     56386
1     55637
3     21608
4     17987
0      8699
5      7746
6      3064
7      1218
8       494
9       192
10      113
11       42
12       35
13        9
20        6
16        3
14        3
15        3
17        1
Name: count, dtype: int64

In [17]:
# recode Household size to categories

size_cat = [
    (skagit_pums['NP'] == 1),
    (skagit_pums['NP'] == 2),
    (skagit_pums['NP'] == 3),
    (skagit_pums['NP'] >= 4)
]
size_cat_labels = ['SIZE1','SIZE2','SIZE3','SIZE4']
skagit_pums['HHSIZE'] = np.select(size_cat, size_cat_labels, default='NA')

skagit_pums['HHSIZE'].value_counts() # freq table

C:\Users\michael.mccarthy\AppData\Local\Temp\ipykernel_5908\770108885.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skagit_pums['HHSIZE'] = np.select(size_cat, size_cat_labels, default='NA')


HHSIZE
SIZE2    3390
SIZE1    2361
NA       1106
SIZE4    1058
SIZE3     816
Name: count, dtype: int64

In [18]:
qa = skagit_pums[skagit_pums['HHSIZE'] == 'NA']
qa['NP'].value_counts()

NP
0    1106
Name: count, dtype: int64

In [19]:
# recode income to categories
skagit_pums['HHINC_22'] = skagit_pums['HINCP'] * 1.042311 # 2022 ADJINC factor

income_cat = [
    (skagit_pums['HHINC_22'] < 25000),
    (skagit_pums['HHINC_22'] >= 25000) & (skagit_pums['HHINC_22'] < 50000),
    (skagit_pums['HHINC_22'] >= 50000) & (skagit_pums['HHINC_22'] < 75000),
    (skagit_pums['HHINC_22'] >= 75000)
]
income_cat_labels = ['INC1','INC2','INC3','INC4']
skagit_pums['INC_CAT'] = np.select(income_cat, income_cat_labels, default='NA')

skagit_pums['INC_CAT'].value_counts() # freq table

C:\Users\michael.mccarthy\AppData\Local\Temp\ipykernel_5908\2991501976.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skagit_pums['HHINC_22'] = skagit_pums['HINCP'] * 1.042311 # 2022 ADJINC factor
C:\Users\michael.mccarthy\AppData\Local\Temp\ipykernel_5908\2991501976.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skagit_pums['INC_CAT'] = np.select(income_cat, income_cat_labels, default='NA')


INC_CAT
INC4    3751
NA      1491
INC2    1295
INC3    1247
INC1     947
Name: count, dtype: int64

In [20]:
# recode Household vehicle to categories

size_cat = [
    (skagit_pums['VEH'] == 0),
    (skagit_pums['VEH'] == 1),
    (skagit_pums['VEH'] == 2),
    (skagit_pums['VEH'] >= 3)
]
size_cat_labels = ['VEH0','VEH1','VEH2','VEH3']
skagit_pums['VEHSIZE'] = np.select(size_cat, size_cat_labels, default='NA')

skagit_pums['VEHSIZE'].value_counts() # freq table

C:\Users\michael.mccarthy\AppData\Local\Temp\ipykernel_5908\1914904437.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skagit_pums['VEHSIZE'] = np.select(size_cat, size_cat_labels, default='NA')


VEHSIZE
VEH2    2964
VEH3    2183
VEH1    1840
NA      1491
VEH0     253
Name: count, dtype: int64

In [21]:
gq = skagit_pums[skagit_pums['TYPEHUGQ'] > 1]
gq['WGTP'].sum()

np.int64(0)

In [22]:
qa = skagit_pums[(skagit_pums['INC_CAT'] == 'NA') & ~(skagit_pums['HHSIZE'] == 'NA')]
qa['TYPEHUGQ'].value_counts()

TYPEHUGQ
3    284
2    101
Name: count, dtype: int64

In [23]:
pums_sizebyinc = pd.crosstab(skagit_pums['HHSIZE'],skagit_pums['INC_CAT'], rownames=['HHSize'],colnames=['HHIncome'])
pums_sizebyinc.to_csv('PUMS_SizebyIncome.csv')

In [24]:
skagit_pums["HH_Persons"] = np.where(skagit_pums["NP"]>4, 4, skagit_pums["NP"])
skagit_pums["HH_Vehicles"] = np.where(skagit_pums["VEH"]>3, 3, skagit_pums["VEH"])
skagit_pums.loc[skagit_pums["HH_Vehicles"] > skagit_pums["HH_Persons"], "HH_Vehicles"] = skagit_pums["HH_Persons"]

C:\Users\michael.mccarthy\AppData\Local\Temp\ipykernel_5908\1623073211.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skagit_pums["HH_Persons"] = np.where(skagit_pums["NP"]>4, 4, skagit_pums["NP"])
C:\Users\michael.mccarthy\AppData\Local\Temp\ipykernel_5908\1623073211.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skagit_pums["HH_Vehicles"] = np.where(skagit_pums["VEH"]>3, 3, skagit_pums["VEH"])


In [25]:
pums_sizebyveh = pd.crosstab(skagit_pums['HH_Persons'],skagit_pums['HH_Vehicles'], rownames=['HHSize'],colnames=['HHVehicles'])
pums_sizebyveh.to_csv('PUMS_SizebyVehicles.csv')

In [31]:
# PUMS 2018-22 5 year person data
pumsp = pd.read_csv('data/PUMS/psam_p53.csv')

In [32]:
# recode workers to cat
# ESR = Employment Status Recode
# from MCCOG: ESR %in% c(1,2,4,5) ~ 1, # Employed or Armed Forces
#             ESR %in% c(3,6) | is.na(ESR) ~ 0 # # Unemployed, Not in Labor Force, or under 16

pumsp['Worker'] = np.where(pumsp['ESR'].isin([1,2,4,5]),1,0)

# Group by serial no, join to HH records
pums_workers = pumsp.groupby(['SERIALNO'])['Worker'].sum()


In [33]:
pums2 = skagit_pums.merge(pums_workers, how="left", on="SERIALNO")

In [34]:

workers_cat = [
    (pums2['Worker'] == 0),
    (pums2['Worker'] == 1),
    (pums2['Worker'] == 2),
    (pums2['Worker'] >= 3)
]
workers_cat_labels = ['WRK0','WRK1','WRK2','WRK3']
pums2['WRK_CAT'] = np.select(workers_cat, workers_cat_labels, default='NA')
pums2.to_csv('PUMS_HHWorkers_ESRecode.csv')

pums_sizebywrk = pd.crosstab(pums2['HHSIZE'],pums2['WRK_CAT'], rownames=['HHSize'],colnames=['HHWorkers'])
pums_sizebywrk.to_csv('PUMS_SizebyWorkers.csv')

In [35]:
pums2['Worker'].value_counts()

Worker
0.0    3195
1.0    2444
2.0    1633
3.0     268
4.0      64
5.0      15
6.0       5
9.0       1
Name: count, dtype: int64

In [36]:
pums2[pums2['TYPEHUGQ'] > 1]

,RT,SERIALNO,DIVISION,PUMA10,PUMA20,REGION,ST,ADJHSG,ADJINC,WGTP,...,WGTP77,WGTP78,WGTP79,WGTP80,HHSIZE,HHINC_22,INC_CAT,VEHSIZE,Worker,WRK_CAT
0,H,2018GQ0003095,9,10200,-9,4,53,1169060,1184371,0,...,0,0,0,0,SIZE1,NaN,NA,NA,1.0,WRK1
1,H,2018GQ0003696,9,10200,-9,4,53,1169060,1184371,0,...,0,0,0,0,SIZE1,NaN,NA,NA,0.0,WRK0
2,H,2018GQ0005667,9,10200,-9,4,53,1169060,1184371,0,...,0,0,0,0,SIZE1,NaN,NA,NA,0.0,WRK0
3,H,2018GQ0005989,9,10200,-9,4,53,1169060,1184371,0,...,0,0,0,0,SIZE1,NaN,NA,NA,0.0,WRK0
4,H,2018GQ0006346,9,10200,-9,4,53,1169060,1184371,0,...,0,0,0,0,SIZE1,NaN,NA,NA,1.0,WRK1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6510,H,2021GQ0151614,9,10200,-9,4,53,1080912,1113261,0,...,0,0,0,0,SIZE1,NaN,NA,NA,1.0,WRK1
6511,H,2021GQ0152502,9,10200,-9,4,53,1080912,1113261,0,...,0,0,0,0,SIZE1,NaN,NA,NA,1.0,WRK1
6512,H,2021GQ0153460,9,10200,-9,4,53,1080912,1113261,0,...,0,0,0,0,SIZE1,NaN,NA,NA,1.0,WRK1
6513,H,2021GQ0156677,9,10200,-9,4,53,1080912,1113261,0,...,0,0,0,0,SIZE1,NaN,NA,NA,1.0,WRK1


In [37]:
from itertools import product

no = [1,2,3,4,5,6,7,8,9,10]
hhsize = [1,2,3,4] 
hhinc = [1,2,3,4]
hhwrk = [0,1,2,3]
tothh = [0]
combinations = list(product(no,hhsize,hhinc,hhwrk,tothh))
test = pd.DataFrame(combinations, columns=['NO','HHSIZE','HHINC','HHWRK','TOTHH'])
test.head()

,NO,HHSIZE,HHINC,HHWRK,TOTHH
0,1,1,1,0,0
1,1,1,1,1,0
2,1,1,1,2,0
3,1,1,1,3,0
4,1,1,2,0,0


## 2020 Decennial Census Blocks

For resolving partial tract coverage in model

In [38]:
dec = Census(api_key,year=2020) # my API key
blocks = dec.pl.get(('GEO_ID','H1_001N','H1_002N','H1_003N','P1_001N','P5_001N','P5_002N','P5_007N'), 
                     geo={'for': 'block:*',
                       'in': 'state:{} county:057'.format(states.WA.fips)})

blocks = pd.DataFrame(blocks)
blocks = blocks.rename(columns={'H1_001N': 'TotHousingUnits','H1_002N':'Occupied','H1_003N':'Vacant','P1_001N':'TotPop','P5_001N':'TotGroupQuarters','P5_002N': 'Institutionalized','P5_007N':'NonInstitutionalized'})

In [39]:
#blocks = blocks[blocks['tract'].isin(['951101','951200'])]

# fix geoid
blocks['GEOID2'] = blocks['GEO_ID'].str.slice(start=9)

In [40]:
blocks.to_csv('2020_blocks_all.csv')